In [ ]:
#import Librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import  KFold , GridSearchCV, train_test_split
from sklearn.ensemble import  RandomForestClassifier

In [ ]:
#Data import
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_features.drop(['sig_id'], axis=1, inplace=True)
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets.drop(['sig_id'], axis=1, inplace=True)
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
test_features.drop(['sig_id'], axis=1, inplace=True)
sub = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
train_features.shape,test_features.shape , train_targets.shape

# EDA

In [ ]:
plt.hist(train_targets.mean())

**#Categorical variables distribution
**

In [ ]:
catList = ['cp_type', 'cp_dose']
countList =  list (set(train_features.columns) - set(catList))

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(5, 5))

for i, ax in enumerate(fig.axes):
    if i < len(catList):
        ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=90)
        sns.countplot(x=catList[i], alpha=0.7, data=train_features, ax=ax)

fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(5, 5))

for i, ax in enumerate(fig.axes):

    ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=90)

    sns.countplot(x='cp_time', alpha=0.7, data=test_features, ax=ax)

In [ ]:
# features data transformation 
#cp_dose column into a numeric value
s= list()
for i in range(23814):
    s.append(train_features['cp_dose'][i][1])
s = [int(item) for item in s]
print(type(s))

train_features['cp_dose'] =s
#cp_type column into a numeric value
s= []
for i in range(23814):
    if train_features['cp_type'][i] == "trt_cp":
        s.append(1)
    else :
        s.append(2)
train_features['cp_type']=s

In [ ]:
# test_features data transformation 
#cp_dose column into a numeric value
m= list()
for i in range(3982):
    m.append(test_features['cp_dose'][i][1])
m = [int(item) for item in m]
print(type(m))
test_features['cp_dose'] =m
#cp_type column into a numeric value
k= []
for i in range(3982):
    if test_features['cp_type'][i] == "trt_cp":
        k.append(1)
    else :
        k.append(2)
test_features['cp_type']=k

In [ ]:
test_features.head()

In [ ]:
train_features

In [ ]:
#Feature significance analysis based on target (target_scored)
train_targets.mean()[train_targets.mean() == train_targets.mean().max()]

In [ ]:
#Renvoie des statistiques de valeur F triées 
def getContinuousVariablesRanking(df, target):
    cont_vars = [i for i in list(df.dtypes[df.dtypes != 'object'].index) if i != target]

    Fvalue_selector = SelectKBest(f_classif, k=len(cont_vars))
    Fvalue_selector.fit_transform(df[cont_vars].fillna(-1), df[target])
    df_Fvalue = pd.DataFrame(Fvalue_selector.scores_,
                             columns=['scaled_importance'])
    # scaling the statistics
    df_Fvalue -= df_Fvalue.min()
    df_Fvalue /= df_Fvalue.max()
    df_Fvalue['columns'] = cont_vars
    df_Fvalue.sort_values(by='scaled_importance', ascending=False, inplace=True)
    
    return df_Fvalue

In [ ]:
features_toptarget_count = pd.concat([train_features[countList], train_targets['nfkb_inhibitor'].astype('str')], axis = 1)

df_value = getContinuousVariablesRanking(features_toptarget_count, 'nfkb_inhibitor')

df_value

In [ ]:
colist = df_value.head(10)['columns']
label = 'nfkb_inhibitor'

for col in colist: 
    
    g = sns.FacetGrid(features_toptarget_count[[col, label]],  hue =label, height = 4, aspect = 1.5) 
    g.map(sns.distplot, col, hist = False, kde_kws = {'shade': True, 'linewidth': 3}).set_axis_labels(col,"density").add_legend()

# Trainning with random forest

In [ ]:
train_features.shape

In [ ]:
#apply PCA to reduce fatures
from sklearn.decomposition import PCA
pca = PCA(n_components= 60)
train_pca_features = pca.fit_transform(train_features)
test_pca_features= pca.fit_transform(test_features)
train_pca_features.shape

In [ ]:
colScoredList = list(train_targets.columns[1:])
colScoredList

In [ ]:
from sklearn.ensemble import  RandomForestClassifier
for label in colScoredList: 
    
    y_train = train_targets[label]
    rf = RandomForestClassifier(class_weight='balanced', max_depth=15, n_estimators=500, n_jobs=-1,random_state=1234)
    rf.fit(train_pca_features, y_train)
    test_features[label] =  rf.predict_proba(test_pca_features)[:,1] 

In [ ]:
submission_test=test_features[['sig_id'] + colList]
submission_test.head()

In [ ]:
submission_test.to_csv('submission.csv', index=False)